# Solution to Exercise 08 - Web Scraping

In today's exercise we're using the Python libraries *BeautifulSoup* and *owlready2* to create an ontology from data scraped from the Web.
[BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/) is a library for extracting data from HTML or XML files by accessing concrete elements in the tree structure.
The ontology (& the web scraping) is foused on extracting data about Pokémon from a wiki-like website called *[Bulbapedia](https://bulbapedia.bulbagarden.net/wiki/Main_Page)*.

## Setup

In [30]:
!pip install bs4
!pip install webdriver-manager
!pip install curl_cffi
!pip install rapidfuzz


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 13.6 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [41]:


from curl_cffi import requests
from curl_cffi.const import CurlHttpVersion  
from bs4 import BeautifulSoup
import time
from rapidfuzz import process, fuzz, utils


In [ ]:
import json
import time
from curl_cffi import requests
from rapidfuzz import process, fuzz, utils

class TechScraper:
    def __init__(self, domain="www.cpubenchmark.net"):
        self.domain = domain
        # Determine if we are looking for CPU or GPU to set page and cache names
        self.is_cpu = "cpu" in domain
        self.mega_page = f"https://{domain}/CPU_mega_page.html" if self.is_cpu else f"https://{domain}/GPU_mega_page.html"
        self.cache_file = "cpu_data.json" if self.is_cpu else "gpu_data.json"
        
        # Load or Scrape
        self.items = self.load_local()
        
        if not self.items:
            print(f"--- No local data found. Starting Scrape for {self.domain} ---")
            self.session = requests.Session()
            self.items = self.scrape()
            if self.items:
                self.save_local()

    def clean_mark(self, value):
        """Safely converts benchmark strings to integers, handling 'NA' or 'Insufficient data'."""
        if value is None:
            return 0
        
        # Convert to string and remove commas
        val_str = str(value).replace(',', '').strip()
        
        # Check if the string is numeric (handles negative signs too)
        if val_str.replace('-', '', 1).isdigit():
            return int(val_str)
        
        # If it's "Insufficient data", "NA", or empty, return 0
        return 0

    def scrape(self):
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/120.0.0.0",
            "Accept": "application/json, text/javascript, */*; q=0.01",
            "X-Requested-With": "XMLHttpRequest",
            "Referer": self.mega_page,
            "Connection": "keep-alive"
        }
        
        try:
            print(f"Warming up session for {self.domain}...")
            self.session.get(self.mega_page, impersonate="chrome120")
            time.sleep(3)
            
            print("Requesting hardware list...")
            data_url = f"https://{self.domain}/data/?_={int(time.time()*1000)}"
            response = self.session.get(data_url, headers=headers, impersonate="chrome120", http_version="v1")
            
            data = response.json().get("data", [])
            if not data:
                print("❌ Server returned empty list.")
                return []
            print(data[0])
            print(f"✅ Success! Found {len(data)} items.")
            
            mark_key = 'cpumark' if self.is_cpu else 'g3dmark'
            
            # Use the new clean_mark function here
            return [
                {
                    "name": x.get('name'), 
                    "mark": self.clean_mark(x.get(mark_key)), 
                    "rank": self.clean_mark(x.get('rank'))
                } 
                for x in data
            ]
        except Exception as e:
            print(f"❌ Scrape failed: {e}")
            return []

    def save_local(self):
        with open(self.cache_file, 'w') as f:
            json.dump(self.items, f)
        print(f"💾 Data saved to {self.cache_file}.")

    def load_local(self):
        try:
            with open(self.cache_file, 'r') as f:
                data = json.load(f)
                
                print(f"📂 Loaded {len(data)} items from {self.cache_file}.")
                return data
        except FileNotFoundError:
            return None

    def get_best_match(self, query):
        if not self.items: return None
        names = [i['name'] for i in self.items]
        res = process.extractOne(query, names, scorer=fuzz.token_set_ratio, processor=utils.default_process)
        return next(i for i in self.items if i['name'] == res[0]) if res and res[1] > 60 else None

    def compare(self, a, b):
        ia, ib = self.get_best_match(a), self.get_best_match(b)
        if not ia or not ib: return "❌ Model not found."
        diff = ((ia['mark'] - ib['mark']) / ib['mark']) * 100
        label = "CPU Mark" if self.is_cpu else "G3D Mark"
        return f"\n🚀 {ia['name']} ({label}: {ia['mark']}) is {abs(diff):.1f}% {'faster' if diff > 0 else 'slower'} than {ib['name']} ({ib['mark']})."

# --- RUN ---

# 1. CPU Section
cpu_tool = TechScraper("www.cpubenchmark.net")
if cpu_tool.items:
    print(cpu_tool.compare("i5-8600", "i9-13900K"))

print("-" * 30)

# 2. GPU Section
gpu_tool = TechScraper("www.videocardbenchmark.net")
if gpu_tool.items:
    print(gpu_tool.compare("RTX 3060", "RTX 4090"))

AttributeError: 'TechScraper' object has no attribute 'items'